In [1]:
import torch
from torch.utils.data import DataLoader,Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, img_as_uint, img_as_float
print(torch.__version__)

1.4.0+cu100


In [2]:
#自定义datasets
class SAR_png_dataset(Dataset): 
    #因为以图像格式读入会快一些，直接读取csv loader很慢
    def __init__(self, root_dir, info_csv,transform=None,transform_label=None): #__init__是初始化该类的一些基础参数 :'E:\\SAR\\exp3'
        self.root_dir = root_dir   #文件目录
        self.transform = transform #变换
        self.transform_label=transform_label
        self.files=pd.read_csv(os.path.join(self.root_dir,info_csv))
        
    def __len__(self):#返回整个数据集的大小files
        return self.files.shape[0]
    def __getitem__(self,index):#根据索引index返回dataset[index]
        real_file = (self.files.iloc[index])['real'] #根据索引index获取该real文件名
        imag_file = (self.files.iloc[index])['imag']
        label_file = (self.files.iloc[index])['label']
        io.use_plugin('matplotlib')
        real=io.imread(os.path.join(self.root_dir,'real_img',real_file))
        real_array=(np.array(real))[:,:,np.newaxis]
        imag=io.imread(os.path.join(self.root_dir,'imag_img',imag_file))
        imag_array=(np.array(imag))[:,:,np.newaxis]
        data=np.concatenate((real_array,imag_array),axis=2)
        label=io.imread(os.path.join(self.root_dir,'label_img',label_file))
        label=2*label-1
        #label=Image.open(os.path.join(self.root_dir,'label_img',label_file))
        #label=np.asarray(label, dtype='float64') / 255
        
        if self.transform:
            data = self.transform(data)#对样本进行变换
        if self.transform_label:
            label=self.transform_label(label)
        return data,label #返回该样本


In [3]:
#预处理
transform_data=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5),(0.5,0.5))])
transform_label=transforms.Compose([transforms.ToTensor()])
train_dataset=  SAR_png_dataset('/home/are/SAR/SAR_exp3/exp3_2/dataset','SAR_datasets_info_train.csv',transform_data,transform_label)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
valid_dataset=  SAR_png_dataset('/home/are/SAR/SAR_exp3/exp3_2/dataset','SAR_datasets_info_valid.csv',transform_data,transform_label)
valid_loader=torch.utils.data.DataLoader(valid_dataset,batch_size=64,shuffle=True)

In [4]:
#模型
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class View(nn.Module):
    def __init__(self,shape):
        super(View,self).__init__()
        self.shape=shape
    def forward(self,x):
        return x.view(*self.shape)
    
class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out        
class SAR_model(nn.Module):
    def __init__(self, block, layers, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(SAR_model, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(2, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        #input:32,512,1,1
        self.generator = nn.Sequential(
            # input is Z, going into a convolution 
            nn.ConvTranspose2d( 512,256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # state size. 256*4*4
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # state size. 128*8*8
            nn.ConvTranspose2d( 128,64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # state size. 64*16*16
            nn.ConvTranspose2d( 64, 1, 5, 3, 1, bias=False),
            nn.Tanh()
            # state size. 1*48*48
        )
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                #nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                #nn.init.constant_(m.weight, 1)
                #nn.init.constant_(m.bias, 0)
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias.data, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x=self.generator(x)
        

        return x

    def forward(self, x):
        return self._forward_impl(x)



In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=SAR_model(block=BasicBlock,layers=[2,2,2,2])
model=model.to(device)
print(model)

SAR_model(
  (conv1): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)

In [6]:
#统计参数
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

13,928,384 total parameters.
13,928,384 training parameters.


In [7]:
learning_rate=5e-4
criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),weight_decay=1e-4,lr=learning_rate,betas=(0.9,0.99))

In [8]:
from tensorboardX import SummaryWriter  #写入tensorboard 路径可更改
writer=SummaryWriter('/home/are/SAR/SAR_exp3/exp3_2/log/res_gan_valid_epoch500_time2')

In [ ]:
save_path='/home/are/SAR/SAR_exp3/exp3_2'
num_epochs=500
train_loss=[]
valid_loss=[]
best_loss=1
dataloader={'train':train_loader,'valid':valid_loader}
dataset={'train':train_dataset,'valid':valid_dataset}
for epoch in range(num_epochs):
    print('epoch:',epoch)
    for phase in ['train','valid']:  
        if phase=='train':
            model.train(True)
        else:
            model.train(False) 
        running_loss=0.0
        for i, data in enumerate(train_loader, 0):
            inputs,label=data
            inputs=inputs.to(device)
            label=label.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss=criterion(outputs, label) 
            if phase=='train':
                loss.backward()
                optimizer.step()
            running_loss+=loss.item()
            del outputs,label


        epoch_loss=running_loss/len(dataset[phase])
        if phase=='train':
            writer.add_scalars('loss_train_val',{'train':epoch_loss},epoch)
            train_loss.append(epoch_loss)
            print('{} epoch {} loss: {:.8f}'.format(epoch,phase,epoch_loss))
        if phase=='valid':
            writer.add_scalars('loss_train_val',{'valid':epoch_loss},epoch)
            valid_loss.append(epoch_loss)
            if epoch_loss<best_loss:
                best_loss=epoch_loss
                best_model_wts=model.state_dict()
            print('{} epoch {} loss: {:.8f}'.format(epoch,phase,epoch_loss))
        print('best valid loss:',best_loss)
    if epoch%50==0 and epoch!=0:
        torch.save(best_model_wts,os.path.join(save_path,'best_weight_res_gan_valid_'+str(epoch)+'.pth'))
        
      
            

torch.save(best_model_wts,os.path.join(save_path,'best_weight_res_gan_valid_epoch500.pth'))
#存储模型
#torch.save(model.load_state_dict(best_model_wts),os.path.join(save_path,'best_model_res_gan_valid_epoch500.pth'))
           
with open(os.path.join(save_path,'train_loss.txt'),'w') as f:
    for line in train_loss:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'valid_loss.txt'),'w') as f:
    for line in valid_loss:
        f.write(str(line)+'\n')  

epoch: 0
0 epoch train loss: 0.00530758
best valid loss: 1
0 epoch valid loss: 0.00403201
best valid loss: 0.0040320072881877425
epoch: 1
1 epoch train loss: 0.00136313
best valid loss: 0.0040320072881877425
1 epoch valid loss: 0.00270764
best valid loss: 0.0027076407708227636
epoch: 2
2 epoch train loss: 0.00052911
best valid loss: 0.0027076407708227636
2 epoch valid loss: 0.00155879
best valid loss: 0.001558790197595954
epoch: 3
3 epoch train loss: 0.00043830
best valid loss: 0.001558790197595954
3 epoch valid loss: 0.00148615
best valid loss: 0.0014861495746299625
epoch: 4
4 epoch train loss: 0.00032983
best valid loss: 0.0014861495746299625
4 epoch valid loss: 0.00121245
best valid loss: 0.0012124530039727688
epoch: 5
5 epoch train loss: 0.00029860
best valid loss: 0.0012124530039727688
5 epoch valid loss: 0.00117542
best valid loss: 0.0011754242470487953
epoch: 6
6 epoch train loss: 0.00029174
best valid loss: 0.0011754242470487953
6 epoch valid loss: 0.00116266
best valid loss: 0

56 epoch valid loss: 0.00039147
best valid loss: 0.00032273945456836375
epoch: 57
57 epoch train loss: 0.00007588
best valid loss: 0.00032273945456836375
57 epoch valid loss: 0.00036205
best valid loss: 0.00032273945456836375
epoch: 58
58 epoch train loss: 0.00007674
best valid loss: 0.00032273945456836375
58 epoch valid loss: 0.00055209
best valid loss: 0.00032273945456836375
epoch: 59
59 epoch train loss: 0.00007733
best valid loss: 0.00032273945456836375
59 epoch valid loss: 0.00030317
best valid loss: 0.00030317077354993674
epoch: 60
60 epoch train loss: 0.00007072
best valid loss: 0.00030317077354993674
60 epoch valid loss: 0.00027356
best valid loss: 0.0002735649049282074
epoch: 61
61 epoch train loss: 0.00006829
best valid loss: 0.0002735649049282074
61 epoch valid loss: 0.00033950
best valid loss: 0.0002735649049282074
epoch: 62
62 epoch train loss: 0.00007313
best valid loss: 0.0002735649049282074
62 epoch valid loss: 0.00041657
best valid loss: 0.0002735649049282074
epoch: 63

110 epoch train loss: 0.00004232
best valid loss: 0.0001596094114938751
110 epoch valid loss: 0.00019058
best valid loss: 0.0001596094114938751
epoch: 111
111 epoch train loss: 0.00004138
best valid loss: 0.0001596094114938751
111 epoch valid loss: 0.00017750
best valid loss: 0.0001596094114938751
epoch: 112
112 epoch train loss: 0.00004133
best valid loss: 0.0001596094114938751
112 epoch valid loss: 0.00015742
best valid loss: 0.00015741955605335533
epoch: 113
113 epoch train loss: 0.00004146
best valid loss: 0.00015741955605335533
113 epoch valid loss: 0.00020417
best valid loss: 0.00015741955605335533
epoch: 114
114 epoch train loss: 0.00004029
best valid loss: 0.00015741955605335533
114 epoch valid loss: 0.00020112
best valid loss: 0.00015741955605335533
epoch: 115
115 epoch train loss: 0.00004153
best valid loss: 0.00015741955605335533
115 epoch valid loss: 0.00022809
best valid loss: 0.00015741955605335533
epoch: 116
116 epoch train loss: 0.00004086
best valid loss: 0.00015741955

In [15]:
torch.save(best_model_wts,os.path.join(save_path,'best_weight_res_gan_valid_epoch500.pth'))

In [5]:
model=SAR_model(block=BasicBlock,layers=[2,2,2,2])
model=model.to(device)
load_path='/home/are/SAR/SAR_exp3/exp3_2/best_weight_res_gan_valid_epoch500.pth'
model.load_state_dict(torch.load(load_path))
model.train(False)

NameError: name 'device' is not defined

torch.Size([64, 2, 120, 540])
torch.Size([64, 1, 48, 48])


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

dataiter = iter(train_loader)
images, labels = dataiter.next()
def imshow(img):
    npimg = np.squeeze(img.numpy())
    #plt.imshow(np.transpose(npimg, (1, 2, 0)))
    print(npimg.shape)
    plt.imshow(npimg)
    
    plt.show()
    
for i in range(32):
    inputs=torch.unsqueeze(images[i,:,:,:],0)
    
    #inp=inputs[:,0,:,:].numpy()
    #inp=np.squeeze(inp)
    #np.savetxt('new_'+str(i)+'.csv', inp, delimiter = ',')  
    inputs=inputs.to(device)
    outputs=model(inputs)
        #outputs=outputs.view((-1,1,120,91)) #[1, 512, 451]
        
    #print(outputs)
    #print(outputs.shape)
    #w=outputs.cpu().detach().numpy()
    #ww= np.squeeze(w)
    #print(ww.shape)
    label=labels[i,:,:,:] #[1, 512, 451]
    imshow(outputs.cpu().detach())
    imshow(label)
    


NameError: name 'torch' is not defined

In [35]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())
    print(parameters)

features.0.weight : torch.Size([64, 2, 3, 3])
Parameter containing:
tensor([[[[ 4.9263e-40,  5.8838e-40,  5.7563e-41],
          [-1.2276e-34,  1.8184e-38,  2.5367e-40],
          [ 1.0157e-39,  3.0807e-40, -2.0890e-40]],

         [[-7.3257e-37,  2.3926e-37,  3.7838e-37],
          [-4.6251e-37, -7.1459e-39, -1.7811e-40],
          [ 1.6163e-37,  2.1067e-40,  2.1171e-39]]],


        [[[ 1.6202e-34,  5.3704e-40, -5.2057e-40],
          [-5.6085e-40, -8.8543e-34,  1.0954e-39],
          [ 5.1050e-40,  1.1442e-39,  9.4326e-35]],

         [[-2.0587e-34, -6.0086e-40, -2.9953e-35],
          [ 2.6514e-33,  1.3645e-39,  1.8771e-36],
          [-1.3276e-39,  1.6238e-39, -9.0727e-41]]],


        [[[-2.2032e-40,  4.1497e-35, -3.3575e-36],
          [ 4.8006e-40,  6.3554e-38, -5.5308e-40],
          [ 4.9950e-36, -2.1975e-40, -1.8891e-35]],

         [[ 5.9007e-40, -1.8512e-33,  3.1042e-40],
          [ 2.3661e-35,  7.9169e-36, -5.4280e-40],
          [ 2.0063e-33,  1.9766e-39,  9.7710e-41]]]

Parameter containing:
tensor([[[[-3.4137e-40, -6.2210e-40, -1.9191e-40],
          [-5.9289e-40,  6.5246e-41,  4.1840e-41],
          [ 5.2780e-40,  2.1351e-40,  6.0395e-40]],

         [[ 3.8129e-41,  6.2396e-41,  4.8332e-41],
          [ 4.9322e-40, -1.4336e-40, -5.9626e-40],
          [ 6.0671e-40, -5.7222e-40,  5.7514e-40]],

         [[ 4.9902e-40,  2.2028e-41, -4.8677e-40],
          [-1.4679e-40,  5.1729e-40, -3.5207e-40],
          [ 5.1241e-40,  1.4426e-41,  1.1763e-40]],

         ...,

         [[-1.8623e-42,  2.0780e-40, -9.0818e-42],
          [-5.9213e-40,  5.6818e-40, -5.5821e-40],
          [ 6.1137e-40,  1.9603e-41, -4.6215e-40]],

         [[-1.6736e-40, -5.2834e-40,  1.4320e-41],
          [ 4.3676e-40, -2.6982e-41, -1.4257e-40],
          [-4.9753e-40, -6.1088e-40, -3.3540e-40]],

         [[ 9.1412e-41, -3.1248e-40, -2.0072e-41],
          [ 5.2271e-40, -3.4805e-40, -4.9176e-40],
          [ 3.6703e-41, -5.5927e-40,  2.4701e-41]]],


        [[[-4.1902e-41,  3.1609

In [43]:
##表示疑惑？为什么直接加模型的part是可以linear不跟前面相关，而自己写是需要相关的
model.classifier[0].in_features=2000
x=torch.rand((32,3,256,256))
y=model(x)
print('y',y.shape)
y1=model.features(x)
print('y1',y1.shape)
y2=model.avgpool(y1)
print('y2',y2.shape)
y2=torch.flatten(y2,1)
print('y2',y2.shape)
print(type(model.classifier))
y3=model.classifier(y2)
print('y3',y3.shape)

part3=nn.Sequential(
            nn.Linear(2000, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),
        )
print(type(part3))
y4=part3(y2)
#y4=nn.Linear(2000, 4096)(y2)
print('y4:',y4.shape)

y torch.Size([32, 1000])
y1 torch.Size([32, 512, 8, 8])
y2 torch.Size([32, 512, 7, 7])
y2 torch.Size([32, 25088])
<class 'torch.nn.modules.container.Sequential'>
y3 torch.Size([32, 1000])
<class 'torch.nn.modules.container.Sequential'>


RuntimeError: size mismatch, m1: [32 x 25088], m2: [2000 x 4096] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:136